In [1]:
# filter out future warnings from numpy on tensorflow calls
import warnings  
import numpy as np

with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow as tf
    

from helpers.tensorflow_graph_in_jupyter import show_graph

## Name Scope

In [2]:
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [3]:
show_graph(tf.get_default_graph())

## Sharing variables: option 1

Variable creation outside function

In [4]:
tf.reset_default_graph()

def relu(X):
    # get the shared variable 
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

# create the shared variable 
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [5]:
show_graph(tf.get_default_graph())

## Sharing variables: option 2

Variable creation inside function

In [6]:
tf.reset_default_graph()

def relu(X):
        # variable is created/accessed here, depending on reuse flag on caller
        threshold = tf.get_variable("threshold", shape=(),
                                    initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")    
relus = []

# variable is created at first iteration, then reused
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index>=1)) as scope:
        relus.append(relu(X))

output = tf.add_n(relus, name="output")

In [7]:
show_graph(tf.get_default_graph())